In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_abs=pd.read_csv('E:/Absenteeism_at_work.csv',sep=';')
df_abs=pd.DataFrame(df_abs)
df_abs

In [ ]:
#checking for any missing data
sns.heatmap(df_abs.isnull())

In [ ]:
df_abs.isnull().sum()

In [ ]:
for i in df_abs.columns:
    print(df_abs[i].value_counts())

In [ ]:
df_abs.dtypes
#checking the datatypes

In [ ]:
#checking the basic info about the attributes and their counts
df_abs.info()

In [ ]:
#checking the maximum and minimum values of each attribute and their percentiles
df_abs.describe()

# Univariate Analysis

In [ ]:
df_abs.hist(figsize = (20,10), layout=(6, 4), bins = 'auto')
plt.tight_layout()
plt.show()

In [ ]:
df_abs.drop(['ID'],axis=1,inplace=True)

In [ ]:
df_abs.columns

In [ ]:
for i in df_abs.columns :
    plt.boxplot(df_abs[i])
    plt.show()

In [ ]:
for i in df_abs.columns:
    sns.violinplot(x=i,data=df_abs)
    plt.tight_layout()
    plt.show()

In [ ]:
#Checking for the distribution of the data
for i in ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons',
       'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target','Son', 'Social drinker',
      'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours']:
    sns.distplot(df_abs[i],bins='auto')
    plt.tight_layout()
    plt.show()

# Bivariate Analysis

In [ ]:
for i in df_abs.columns:
    sns.barplot(df_abs['Absenteeism time in hours'],df_abs[i])
    plt.tight_layout()
    plt.show()

In [ ]:
for i in df_abs.columns:
    sns.catplot(x=i,y='Absenteeism time in hours',data=df_abs)
    plt.tight_layout()
    plt.show()

In [ ]:
#Checking the correlation between the attributes
df_abs.corr()

In [ ]:
plt.figure(figsize=(25,12))
sns.heatmap(df_abs.corr(),cmap='coolwarm',annot=True)
plt.show()

In [ ]:
df_abs.drop(['Month of absence', 'Day of the week', 'Seasons','Hit target','Weight', 'Height'],axis=1,inplace = True)

In [ ]:
df_abs.columns

In [ ]:
df_abs['Absenteeism']=["Late" if period<=0 else
                       'Very late'if 0<=period<=10 else 
                       "Not acceptable late" if period>=10 else None
                      for period in list (df_abs['Absenteeism time in hours'].values)]

In [ ]:
df_abs['Absenteeism']

In [ ]:
df_abs.columns

In [ ]:
import sklearn
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df_abs['Absenteeism']=le.fit_transform(df_abs['Absenteeism'])

In [ ]:
df_abs.drop(['Absenteeism time in hours'],axis=1,inplace = True)

# Multivariate Analysis

In [ ]:
sns.pairplot(df_abs)

In [ ]:
#Cheking the skewness of data
df_abs.skew()

In [ ]:
#removing skewness using log transform
for col in ['Age']:
        df_abs[col]=np.log1p(df_abs[col])

In [ ]:
df_abs.skew()
#reduce skewness

In [ ]:
#removing outliers using zscore
from scipy.stats import zscore
zscore=np.abs(zscore(df_abs))
print(zscore)
print(np.where(zscore>3))
df_abs_new=df_abs[(zscore<3).all(axis=1)]

In [ ]:
print('Before removing outliers:',df_abs.shape)
print('After removing outliers:',df_abs_new.shape)

In [ ]:
#separating into input and output variables
df_x=df_abs_new.drop(columns=['Absenteeism'])
y=pd.DataFrame(df_abs_new['Absenteeism'])

In [ ]:
df_x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(df_x)
x=pd.DataFrame(x,columns=df_x.columns)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB                                                                                                                   
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_curve,auc

In [ ]:
max_acc_scr=0
for r_state in range(42,101):
    x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=r_state,test_size=0.20)
    model=[LogisticRegression(),SVC(),DecisionTreeClassifier(),KNeighborsClassifier(),GaussianNB(),RandomForestClassifier()]
    for m in model:
        m.fit(x_train,y_train)
        m.score(x_train,y_train)
        predm=m.predict(x_test)
        accscr=accuracy_score(y_test,predm)
        conmatrix=confusion_matrix(y_test,predm)
        classreport=classification_report(y_test,predm)
        print("accuracy score of",m,'is:')
        print('accuracy score corresponding to random state:',r_state,'is:',accscr)
        print('confusion matrix corresponding to random state:',r_state,'is:',conmatrix)
        print('classification report corresponding to random state:',r_state,'is:',classreport)
        if accscr>max_acc_scr:
            max_acc_scr=accscr
            final_r_state=r_state
            final_model=m
        print()

In [ ]:
print("Max accuracy Score corresponding to random state",final_r_state,"is",max_acc_scr,'and model is',final_model)

In [ ]:
LR=LogisticRegression()
SV=SVC()
DT=DecisionTreeClassifier()
KNN=KNeighborsClassifier()
GNB=GaussianNB()
RFC=RandomForestClassifier()

In [ ]:
models=[]
models.append(('LogisticRegression',LR))
models.append(('SVC',SV))
models.append(('DecisionTreeClassifier',DT))
models.append(('KNeighborsClassifier',KNN))
models.append(('GaussianNB',GNB))
models.append(('RandomForestClassifier',RFC))

In [ ]:
Model=[]
score=[]
cvs=[]
rocscore=[]
for name,model in models:
    print('***********',name,'***********')
    print('\n')
    Model.append(name)
    model.fit(x_train,y_train)
    print(model)
    pre=model.predict(x_test)
    print('\n')
    AS=accuracy_score(y_test,pre)
    print("Accuracy_score=",AS)
    score.append(AS*100)
    print('\n')
    false_positive_rate,true_positive_rate, thresholds = roc_curve(y_test,pre,pos_label=1)
    roc_auc= auc(false_positive_rate,true_positive_rate)
    print("roc_auc_score=",roc_auc)
    rocscore.append(roc_auc*100)
    print('\n')
    print('classification_report\n',classification_report(y_test,pre))
    print('\n')
    cm=confusion_matrix(y_test,pre)
    print(cm)
    print('\n')
    plt.figure(figsize=(10,40))
    plt.subplot(911)
    plt.title(name)
    print(sns.heatmap(cm,annot=True))
    plt.subplot(912)
    plt.title(name)
    plt.plot(false_positive_rate,true_positive_rate,label='AUC= %0.2f'% roc_auc)
    plt.plot([0,1],[0,1],'r--')
    plt.legend(loc='lower right')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    print('\n\n')   

In [ ]:
import sklearn
from sklearn.model_selection import cross_val_score
score=cross_val_score(sklearn.svm.SVC(),x,y,cv=5)
print(score)
print(score.mean())
print(score.std())

In [ ]:
#Trying to use Gradient Boosting Technique
#For getting best set of parameters, using GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
parameters={'learning_rate':[0.0001,0.001,0.01,0.1,1],'n_estimators':[50,100,150,200,250,300,350,400,450,500]}
clf=GridSearchCV(gbc,parameters,cv=5)
clf.fit(x,y)
clf.best_params_

In [ ]:
#Using cross val score to check the mean score and standard deviation
gbc=GradientBoostingClassifier(learning_rate=0.0001,n_estimators=50)
print("Mean score for Gradient Boosting Classifier:",cross_val_score(gbc,x,y,cv=5).mean())
print('Standard Deviation in score for Gradient Boosting Classifier:',cross_val_score(gbc,x,y,cv=5).std())

In [ ]:
max_acc_scr=0
for r_state in range(42,101):
    x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=r_state,test_size=0.20)
    model=[GradientBoostingClassifier(learning_rate=0.1,n_estimators=200)]
    for m in model:
        m.fit(x_train,y_train)
        m.score(x_train,y_train)
        predm=m.predict(x_test)
        accscr=accuracy_score(y_test,predm)
        conmatrix=confusion_matrix(y_test,predm)
        classreport=classification_report(y_test,predm)
        print("accuracy score of",m,'is:')
        print('accuracy score corresponding to random state:',r_state,'is:',accscr)
        print('confusion matrix corresponding to random state:',r_state,'is:',conmatrix)
        print('classification report corresponding to random state:',r_state,'is:',classreport)
        if accscr>max_acc_scr:
            max_acc_scr=accscr
            final_r_state=r_state
            final_model=m
        print()

In [ ]:
print("Max accuracy Score corresponding to random state",final_r_state,"is",max_acc_scr)

In [ ]:
#We tried all methods and till now SVC is the best option 
#Random state corresponding to highest r2 score is 63
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=63,test_size=0.20)
sv=SVC()
sv.fit(x_train,y_train)
predm=sv.predict(x_test)
accscr=accuracy_score(y_test,predm)
print("Mean score for SVC:",cross_val_score(sv,x,y,cv=5).mean())
print('Standard Deviation in score for SVC:',cross_val_score(sv,x,y,cv=5).std())
print("Accuracy score is:",accuracy_score(y_test,predm))

In [ ]:
#Saving the modelas a pickle in a file
from sklearn.externals import joblib
joblib.dump(sv,"Project16_Absenteeism.pkl")